In [22]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader

In [3]:
data = np.array([100, 110, 120, 125, 130, 140, 150, 155, 160, 170, 180, 190])

In [5]:
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(data[i + seq_length])
    return sequences, targets

In [6]:
seq_length = 5
sequences, targets = create_sequences(data, seq_length)

In [7]:
# Convert sequences and targets to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.float32)
targets = torch.tensor(targets, dtype=torch.float32)

In [8]:
# Split data into training and testing sets
train_size = int(0.8 * len(sequences))
train_sequences, test_sequences = sequences[:train_size], sequences[train_size:]
train_targets, test_targets = targets[:train_size], targets[train_size:]

In [9]:
class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Get the last output in the sequence
        return out

input_size = 1  # Sales data is univariate
hidden_size = 64
num_layers = 1

model = LSTMPredictor(input_size, hidden_size, num_layers)

In [13]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [14]:
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(train_sequences.unsqueeze(2))
    loss = criterion(outputs, train_targets)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Training complete!')


Epoch [10/1000], Loss: 8543.8770
Epoch [20/1000], Loss: 7596.9512
Epoch [30/1000], Loss: 6709.9351
Epoch [40/1000], Loss: 5869.7002
Epoch [50/1000], Loss: 5116.1304
Epoch [60/1000], Loss: 4438.6484
Epoch [70/1000], Loss: 3835.0579
Epoch [80/1000], Loss: 3300.3003
Epoch [90/1000], Loss: 2828.8174
Epoch [100/1000], Loss: 2415.1301
Epoch [110/1000], Loss: 2053.9814
Epoch [120/1000], Loss: 1740.3649
Epoch [130/1000], Loss: 1469.5275
Epoch [140/1000], Loss: 1236.9838
Epoch [150/1000], Loss: 1038.5057
Epoch [160/1000], Loss: 870.1417
Epoch [170/1000], Loss: 728.2207
Epoch [180/1000], Loss: 609.3572
Epoch [190/1000], Loss: 510.4558
Epoch [200/1000], Loss: 428.7119
Epoch [210/1000], Loss: 361.6016
Epoch [220/1000], Loss: 306.8807
Epoch [230/1000], Loss: 262.5679
Epoch [240/1000], Loss: 226.9305
Epoch [250/1000], Loss: 198.4697
Epoch [260/1000], Loss: 175.8987
Epoch [270/1000], Loss: 158.1223
Epoch [280/1000], Loss: 144.2212
Epoch [290/1000], Loss: 133.4260
Epoch [300/1000], Loss: 125.1010
Epoc

In [15]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(test_sequences.unsqueeze(2))
    mse = criterion(test_outputs, test_targets)
    print(f'Mean Squared Error on Test Data: {mse:.4f}')

    # Convert the test_outputs to a NumPy array
    predictions = test_outputs.numpy()
    print('Predicted Sales:', predictions)

Mean Squared Error on Test Data: 925.0060
Predicted Sales: [[154.99991]
 [154.9999 ]]


m:\disco m\python\pythonprojects\pytorch\env_pytorch\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [17]:
test_targets

tensor([180., 190.])

In [18]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(train_sequences.unsqueeze(2))
    mse = criterion(test_outputs, test_targets)
    print(f'Mean Squared Error on Test Data: {mse:.4f}')

    # Convert the test_outputs to a NumPy array
    predictions = test_outputs.numpy()
    print('Predicted Sales:', predictions)

Mean Squared Error on Test Data: 925.0062
Predicted Sales: [[154.9999 ]
 [154.9999 ]
 [154.9999 ]
 [154.9999 ]
 [154.99991]]


m:\disco m\python\pythonprojects\pytorch\env_pytorch\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


***

In [20]:
data = {
    'Price': [100, 105, 110, 120, 115, 112, 108, 109, 105, 102],
    'Label': [0, 0, 1, 1, 1, 0, 0, 1, 1, 0]
}

df = pd.DataFrame(data)

In [23]:
class PriceLabelDataset(Dataset):
    def __init__(self, data, sequence_length=3):
        self.data = data.values  # Convert to numpy array
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        prices = self.data[idx:idx + self.sequence_length, 0]
        label = self.data[idx + self.sequence_length, 1]
        return torch.FloatTensor(prices), torch.LongTensor([label])

# Create the dataset
dataset = PriceLabelDataset(df, sequence_length=3)

# Create a data loader
batch_size = 2  # Adjust as needed
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [25]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x.view(x.size(0), -1, 1), (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Create the model
input_size = 1
hidden_size = 8
num_layers = 2
output_size = 2
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

In [26]:
import torch.optim as optim

# Define hyperparameters
learning_rate = 0.001
num_epochs = 100

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for prices, labels in loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(prices)
        
        # Calculate loss
        loss = criterion(outputs, labels.view(-1))
        
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(loader):.4f}")

print("Training finished.")


Epoch [1/100], Loss: 0.7260
Epoch [2/100], Loss: 0.7233
Epoch [3/100], Loss: 0.7218
Epoch [4/100], Loss: 0.7206
Epoch [5/100], Loss: 0.7195
Epoch [6/100], Loss: 0.7184
Epoch [7/100], Loss: 0.7174
Epoch [8/100], Loss: 0.7164
Epoch [9/100], Loss: 0.7154
Epoch [10/100], Loss: 0.7145
Epoch [11/100], Loss: 0.7135
Epoch [12/100], Loss: 0.7126
Epoch [13/100], Loss: 0.7117
Epoch [14/100], Loss: 0.7109
Epoch [15/100], Loss: 0.7100
Epoch [16/100], Loss: 0.7091
Epoch [17/100], Loss: 0.7083
Epoch [18/100], Loss: 0.7074
Epoch [19/100], Loss: 0.7066
Epoch [20/100], Loss: 0.7058
Epoch [21/100], Loss: 0.7051
Epoch [22/100], Loss: 0.7043
Epoch [23/100], Loss: 0.7036
Epoch [24/100], Loss: 0.7029
Epoch [25/100], Loss: 0.7023
Epoch [26/100], Loss: 0.7017
Epoch [27/100], Loss: 0.7011
Epoch [28/100], Loss: 0.7005
Epoch [29/100], Loss: 0.7000
Epoch [30/100], Loss: 0.6996
Epoch [31/100], Loss: 0.6992
Epoch [32/100], Loss: 0.6988
Epoch [33/100], Loss: 0.6984
Epoch [34/100], Loss: 0.6980
Epoch [35/100], Loss: 0